In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [2]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [3]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler


In [4]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [5]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    plt.plot(y_test, label='Actual')
    plt.plot(lstm_predictions, label='Predicted')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig(f'Bitcoin_Models/actual_vs_predicted_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history.history['loss'], label='Training Loss')
    plt.plot(loss_history.history['val_loss'], label='Validation Loss')
    plt.title('Epoch Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'Bitcoin_Models/loss_curve_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    return model, rmse


In [6]:
symbol = 'BTC-USD'
start_date = '2020-01-01'
end_date = '2024-01-01'

In [8]:
epochs_range = [50, 100]
batch_sizes = [4,8]
sequence_lengths = [25,60,80]
units_range = [50, 100]
layers_range = [2, 3]

In [7]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [9]:
results = []

In [10]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})


Training model with epochs=50, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/50

263/263 [==============================] - 25s 49ms/step - loss: 0.0084 - val_loss: 7.2595e-04
Epoch 2/50
  3/263 [..............................] - ETA: 10s - loss: 0.0039

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


263/263 [==============================] - 9s 34ms/step - loss: 0.0033 - val_loss: 0.0013
Epoch 3/50
263/263 [==============================] - 10s 37ms/step - loss: 0.0027 - val_loss: 7.3448e-04
Epoch 4/50
263/263 [==============================] - 13s 49ms/step - loss: 0.0018 - val_loss: 3.7599e-04
Epoch 5/50
263/263 [==============================] - 11s 40ms/step - loss: 0.0015 - val_loss: 7.8180e-04
Epoch 6/50
263/263 [==============================] - 10s 38ms/step - loss: 0.0015 - val_loss: 2.8015e-04
Epoch 7/50
263/263 [==============================] - 11s 41ms/step - loss: 8.7624e-04 - val_loss: 2.0500e-04
Epoch 8/50
263/263 [==============================] - 11s 44ms/step - loss: 9.0328e-04 - val_loss: 3.9897e-04
Epoch 9/50
263/263 [==============================] - 13s 51ms/step - loss: 7.4628e-04 - val_loss: 3.9562e-04
Epoch 10/50
263/263 [==============================] - 11s 44ms/step - loss: 7.7703e-04 - val_loss: 2.7338e-04
Epoch 11/50
263/263 [========================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


263/263 [==============================] - 13s 50ms/step - loss: 0.0051 - val_loss: 7.9264e-04
Epoch 3/50
263/263 [==============================] - 12s 46ms/step - loss: 0.0031 - val_loss: 5.6476e-04
Epoch 4/50
263/263 [==============================] - 11s 43ms/step - loss: 0.0020 - val_loss: 8.6673e-04
Epoch 5/50
263/263 [==============================] - 13s 48ms/step - loss: 0.0016 - val_loss: 3.4255e-04
Epoch 6/50
263/263 [==============================] - 15s 57ms/step - loss: 0.0015 - val_loss: 3.7046e-04
Epoch 7/50
263/263 [==============================] - 15s 55ms/step - loss: 0.0014 - val_loss: 2.4282e-04
Epoch 8/50
263/263 [==============================] - 13s 50ms/step - loss: 0.0010 - val_loss: 3.4569e-04
Epoch 9/50
263/263 [==============================] - 13s 50ms/step - loss: 9.1475e-04 - val_loss: 4.6918e-04
Epoch 10/50
263/263 [==============================] - 12s 46ms/step - loss: 9.0382e-04 - val_loss: 6.6192e-04
Epoch 11/50
263/263 [===========================

...

132/132 [==============================] - 19s 142ms/step - loss: 0.0044 - val_loss: 8.4905e-04
Epoch 3/100
132/132 [==============================] - 18s 139ms/step - loss: 0.0036 - val_loss: 0.0012
Epoch 4/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0033 - val_loss: 9.2036e-04
Epoch 5/100
132/132 [==============================] - 19s 142ms/step - loss: 0.0020 - val_loss: 6.5331e-04
Epoch 6/100
132/132 [==============================] - 18s 140ms/step - loss: 0.0019 - val_loss: 7.7104e-04
Epoch 7/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0017 - val_loss: 4.3222e-04
Epoch 8/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0015 - val_loss: 4.0717e-04
Epoch 9/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0012 - val_loss: 3.6301e-04
Epoch 10/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0011 - val_loss: 4.0757e-04
Epoch 11/100
132/132 [=====================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


132/132 [==============================] - 22s 165ms/step - loss: 0.0027 - val_loss: 7.8588e-04
Epoch 3/100
132/132 [==============================] - 21s 159ms/step - loss: 0.0027 - val_loss: 9.3593e-04
Epoch 4/100
132/132 [==============================] - 21s 155ms/step - loss: 0.0014 - val_loss: 4.8886e-04
Epoch 5/100
132/132 [==============================] - 20s 154ms/step - loss: 0.0015 - val_loss: 4.3051e-04
Epoch 6/100
132/132 [==============================] - 21s 158ms/step - loss: 0.0014 - val_loss: 3.3468e-04
Epoch 7/100
132/132 [==============================] - 21s 156ms/step - loss: 0.0012 - val_loss: 3.2999e-04
Epoch 8/100
132/132 [==============================] - 21s 157ms/step - loss: 0.0011 - val_loss: 2.0972e-04
Epoch 9/100
132/132 [==============================] - 21s 161ms/step - loss: 8.9038e-04 - val_loss: 3.0849e-04
Epoch 10/100
132/132 [==============================] - 20s 154ms/step - loss: 7.9675e-04 - val_loss: 1.7959e-04
Epoch 11/100
132/132 [=========

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


132/132 [==============================] - 19s 141ms/step - loss: 0.0044 - val_loss: 0.0012
Epoch 3/100
132/132 [==============================] - 19s 142ms/step - loss: 0.0031 - val_loss: 8.8607e-04
Epoch 4/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0031 - val_loss: 8.6377e-04
Epoch 5/100
132/132 [==============================] - 19s 143ms/step - loss: 0.0018 - val_loss: 4.5002e-04
Epoch 6/100
132/132 [==============================] - 19s 142ms/step - loss: 0.0017 - val_loss: 3.6849e-04
Epoch 7/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0016 - val_loss: 6.5273e-04
Epoch 8/100
132/132 [==============================] - 19s 142ms/step - loss: 0.0013 - val_loss: 2.8394e-04
Epoch 9/100
132/132 [==============================] - 19s 140ms/step - loss: 0.0010 - val_loss: 4.9188e-04
Epoch 10/100
132/132 [==============================] - 19s 141ms/step - loss: 0.0011 - val_loss: 5.3700e-04
Epoch 11/100
132/132 [=====================

In [11]:
for result in results:
    print(result)

{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 947.3022114119542}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 674.5037989615822}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 655.7679438763513}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 658.2745992730383}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 2, 'rmse': 676.491914964014}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 3, 'rmse': 805.7078025252819}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 2, 'rmse': 741.108645534591}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 3, 'rmse': 719.6021348839267}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 80, 'units': 50, 'layers': 2, 'rmse': 701.3365197432306}
{'epochs': 50, 'batch_size